In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Load from baseline module data
train = pd.read_csv("../baseline_prophet_forecast/data/train.csv")
features = pd.read_csv("../baseline_prophet_forecast/data/features.csv")
stores = pd.read_csv("../baseline_prophet_forecast/data/stores.csv")

df = (
    train
    .merge(features, on=["Store", "Date", "IsHoliday"])
    .merge(stores, on="Store")
)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Dept", "Date"])
df.head()


In [ ]:
store = 1
dept = 1

ts = (
    df[(df["Store"] == store) & (df["Dept"] == dept)]
    [["Date", "Weekly_Sales", "IsHoliday"]]
    .rename(columns={"Date": "ds", "Weekly_Sales": "y"})
    .sort_values("ds")
    .reset_index(drop=True)
)

ts.head()


In [ ]:
np.random.seed(42)

base_price = 10.0

# Simulate price with small noise
ts["price"] = base_price + np.random.normal(loc=0, scale=0.8, size=len(ts))

# Cap prices to a reasonable range
ts["price"] = ts["price"].clip(lower=6.0, upper=14.0)

# Mark promotions when price is significantly below base
ts["promo"] = (ts["price"] < (base_price - 0.75)).astype(int)

ts[["ds", "y", "price", "promo"]].head()


In [ ]:
# Remove non-positive sales just in case
ts = ts[ts["y"] > 0].copy()

ts["log_y"] = np.log(ts["y"])
ts["log_price"] = np.log(ts["price"])

X = ts[["log_price", "promo"]]
X = sm.add_constant(X)
y = ts["log_y"]

model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
elasticity = model.params["log_price"]   # should be negative
promo_coef = model.params["promo"]

promo_uplift = np.exp(promo_coef) - 1   # convert log-effect to % uplift

print(f"Estimated price elasticity: {elasticity:.3f}")
print(f"Estimated promo uplift: {promo_uplift * 100:.1f}%")


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(ts["price"], ts["y"], alpha=0.6)
plt.xlabel("Price")
plt.ylabel("Weekly Sales")
plt.title(f"Demand Curve — Store {store}, Dept {dept}")
plt.tight_layout()
plt.savefig("images/demand_curve_store1_dept1.png")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
colors = np.where(ts["promo"] == 1, "orange", "blue")

plt.scatter(ts["price"], ts["y"], c=colors, alpha=0.7)
plt.xlabel("Price")
plt.ylabel("Weekly Sales")
plt.title(f"Demand Curve with Promo Highlight — Store {store}, Dept {dept}")
plt.tight_layout()
plt.savefig("images/demand_curve_promo_store1_dept1.png")
plt.show()


Interpretation
• The coefficient on log_price represents price elasticity: a value of, for example, –1.3 means a 1% price increase is associated with a 1.3% decrease in demand.
• The promo coefficient (converted via exp(coef) - 1) represents promo uplift: e.g., a 0.3 value corresponds to roughly 35% higher demand during promo weeks.
• This model is intentionally simple but reflects the core patterns pricing and merchandising teams care about: how sensitive a specific store–department’s demand is to price changes and promotions.